In [20]:


import pandas as pd
import numpy as np
from model_team14 import select_features

## select imputed & transformed data
X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
threshold=0.3
criteria=None

df_feature=pd.read_csv('../data/df_feature.csv')

selected_features=list(df_feature[df_feature.select==1]['variable'])

def get_data(y_type, test_year, features):
    X_train=X[features][:-(test_year*12)]
    y_train=y[y_type][:-(test_year*12)]           
    X_test=X[features][-(test_year*12):]   
    y_test=y[y_type][-(test_year*12):]

    return X_train, y_train, X_test, y_test

In [21]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)
# Timesteps will define how many Elements we have
# TIME_STEPS = 5

# X_train, y_train = create_dataset(X_train, y_train, TIME_STEPS)
# X_test, y_test = create_dataset(X_test, y_test, TIME_STEPS)

# print(X_train.shape)

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

def grid_search(trainX, trainy, testX, testy, y_type, epochs=[15,20], batch_sizes=[12,64], time_steps=[5]):
    results = []
    best_auc = 0
    best_params = None

    for time_step in time_steps:
        X_train, y_train = create_dataset(trainX, trainy, time_step)
        X_test, y_test = create_dataset(testX, testy, time_step)
        loss_type = 'categorical_crossentropy' if y_type == 'y_agg' else 'binary_crossentropy'

        for num_epochs in epochs:
            for batch_size in batch_sizes:
                # create the model
                model = Sequential()
                model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
                # model.add(Dropout(0.2))
                model.add(Dense(1, activation='sigmoid'))

                model.compile(loss=loss_type, optimizer='adam', metrics=['AUC'])
                model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)
                print(model.summary())

                # Final evaluation of the model
                scores = model.evaluate(X_test, y_test, verbose=0)
                auc = scores[1]*100
                print("AUC: %.2f%%" % (auc))

                current_result = {
                    'time_steps': time_step,
                    'num_epochs': num_epochs,
                    'batch_size': batch_size,
                    'AUC': "%.2f%%" % (auc)
                }
                results.append(current_result)

                if auc > best_auc:
                    best_auc = auc
                    best_params = current_result

    print("Best AUC: ", best_params)
    return results


In [23]:
y_type = 'y_agg'
X_train, y_train, X_test, y_test = get_data(y_type, 8, selected_features)

print(grid_search(X_train, y_train, X_test, y_test, y_type))

Epoch 1/15


/Users/richardlieu/.local/share/virtualenvs/milestoneII-GYt9I6pr/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


41/41 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - auc: 0.6156
Epoch 2/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6522
Epoch 3/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6260
Epoch 4/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6560
Epoch 5/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6529
Epoch 6/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6690
Epoch 7/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6743
Epoch 8/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6798
Epoch 9/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6703
Epoch 10/15
41/41 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - auc: 0.6586
Epoch 11/15
41/41 [=====================

In [24]:
y_type = 'y_nber'
X_train, y_train, X_test, y_test = get_data(y_type, 8, selected_features)

print(grid_search(X_train, y_train, X_test, y_test, y_type))

Epoch 1/15
41/41 [==============================] - 1s 2ms/step - loss: 0.4232 - auc: 0.7380
Epoch 2/15
41/41 [==============================] - 0s 1ms/step - loss: 0.2199 - auc: 0.9214
Epoch 3/15
41/41 [==============================] - 0s 2ms/step - loss: 0.1997 - auc: 0.9405
Epoch 4/15
41/41 [==============================] - 0s 1ms/step - loss: 0.1756 - auc: 0.9632
Epoch 5/15
41/41 [==============================] - 0s 2ms/step - loss: 0.1576 - auc: 0.9703
Epoch 6/15
41/41 [==============================] - 0s 2ms/step - loss: 0.1678 - auc: 0.9615
Epoch 7/15
41/41 [==============================] - 0s 1ms/step - loss: 0.1509 - auc: 0.9706
Epoch 8/15
41/41 [==============================] - 0s 2ms/step - loss: 0.1352 - auc: 0.9770
Epoch 9/15
41/41 [==============================] - 0s 5ms/step - loss: 0.1279 - auc: 0.9804
Epoch 10/15
41/41 [==============================] - 0s 2ms/step - loss: 0.1160 - auc: 0.9833
Epoch 11/15
41/41 [==============================] - 0s 1ms/step - lo

In [25]:
y_type = 'y_oecd'
X_train, y_train, X_test, y_test = get_data(y_type, 8, selected_features)

print(grid_search(X_train, y_train, X_test, y_test, y_type))

Epoch 1/15
41/41 [==============================] - 1s 2ms/step - loss: 0.5895 - auc: 0.7744
Epoch 2/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4892 - auc: 0.8458
Epoch 3/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4749 - auc: 0.8457
Epoch 4/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4547 - auc: 0.8595
Epoch 5/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4385 - auc: 0.8682
Epoch 6/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4425 - auc: 0.8651
Epoch 7/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4267 - auc: 0.8759
Epoch 8/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4192 - auc: 0.8816
Epoch 9/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4298 - auc: 0.8726
Epoch 10/15
41/41 [==============================] - 0s 2ms/step - loss: 0.4064 - auc: 0.8872
Epoch 11/15
41/41 [==============================] - 0s 2ms/step - lo